In [6]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
# Cell 2: Data Loading and Preprocessing Functions
# ==============================================================================

def load_data(filepath):
    """
    Loads the restaurant dataset from a specified CSV file.

    Args:
        filepath (str): The path to the dataset CSV file.

    Returns:
        pandas.DataFrame: The loaded dataset.
    """
    try:
        # Use the name of the uploaded file
        df = pd.read_csv("C:\\Users\Admin\Downloads\Dataset .csv", encoding='latin-1')
        print(f"Dataset '{filepath}' loaded successfully.")
        return df
    except FileNotFoundError:
        print(f"Error: The file at {filepath} was not found.")
        return None

def preprocess_data(df):
    """
    Cleans and preprocesses the restaurant DataFrame.

    Args:
        df (pandas.DataFrame): The raw restaurant DataFrame.

    Returns:
        pandas.DataFrame: The cleaned and preprocessed DataFrame.
    """
    df.dropna(subset=['Cuisines'], inplace=True)
    df['Average Cost for two'] = df['Average Cost for two'].replace(0, np.nan)
    df['Average Cost for two'] = df.groupby('Price range')['Average Cost for two'].transform(
        lambda x: x.fillna(x.median())
    )
    if df['Average Cost for two'].isnull().any():
        df['Average Cost for two'].fillna(df['Average Cost for two'].median(), inplace=True)
    df['Cuisines'] = df['Cuisines'].str.lower().str.replace(', ', ' ')
    relevant_cols = ['Restaurant Name', 'Cuisines', 'Price range', 'Aggregate rating', 'Votes', 'City']
    df_relevant = df[relevant_cols].copy()
    df_relevant['Price_Tag'] = 'price_range_' + df_relevant['Price range'].astype(str)
    df_relevant['tags'] = df_relevant['Cuisines'] + ' ' + df_relevant['City'].str.lower() + ' ' + df_relevant['Price_Tag']
    print("Data preprocessing complete.")
    return df_relevant

In [9]:
#Cell 3: Feature Engineering and Recommendation Functions
# ==============================================================================

def create_feature_matrix(dataframe):
    """
    Creates a TF-IDF feature matrix from the 'tags' column.

    Args:
        dataframe (pandas.DataFrame): The preprocessed DataFrame with a 'tags' column.

    Returns:
        tuple: A tuple containing the TF-IDF vectorizer and the TF-IDF matrix.
    """
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(dataframe['tags'])
    print("TF-IDF feature matrix created.")
    return tfidf_vectorizer, tfidf_matrix

def get_recommendations(user_preference, df, vectorizer, tfidf_matrix, top_n=10):
    """
    Generates restaurant recommendations based on user preference.

    Args:
        user_preference (str): A string describing the user's preferences.
        df (pandas.DataFrame): The preprocessed DataFrame of restaurants.
        vectorizer (TfidfVectorizer): The fitted TF-IDF vectorizer.
        tfidf_matrix (scipy.sparse.matrix): The TF-IDF feature matrix.
        top_n (int): The number of recommendations to return.

    Returns:
        pandas.DataFrame: A DataFrame with the top N recommendations.
    """
    user_vec = vectorizer.transform([user_preference.lower()])
    cosine_similarities = cosine_similarity(user_vec, tfidf_matrix).flatten()
    top_indices = cosine_similarities.argsort()[-top_n:][::-1]
    recommendations = df.iloc[top_indices].copy()
    recommendations['Similarity'] = cosine_similarities[top_indices]
    output_cols = ['Restaurant Name', 'Cuisines', 'City', 'Aggregate rating', 'Votes', 'Similarity']
    return recommendations[output_cols]

In [ ]:
# 1. Load and preprocess the data
dataset_path = 'Dataset .csv'
restaurant_df = load_data(dataset_path)

if restaurant_df is not None:
    # 2. Preprocess the data and create the feature matrix
    processed_df = preprocess_data(restaurant_df)
    vectorizer, matrix = create_feature_matrix(processed_df)

    # 3. Start the interactive recommendation session
    print("\n" + "="*40)
    print("--- Restaurant Recommendation System ---")
    print("="*40)
    print("Type 'quit' or 'exit' to stop.")

    while True:
        # Get user input
        user_query = input("\nWhat are you craving? (e.g., 'Japanese Sushi in Mandaluyong City'): ")

        # Check if the user wants to exit
        if user_query.lower() in ['quit', 'exit']:
            print("\nThank you for using the recommendation system. Goodbye!")
            break

        # Generate and display recommendations
        recommended_restaurants = get_recommendations(
            user_preference=user_query,
            df=processed_df,
            vectorizer=vectorizer,
            tfidf_matrix=matrix,
            top_n=5  # You can change the number of recommendations here
        )

        print(f"\nTop 5 Recommended Restaurants for '{user_query}':")
        if recommended_restaurants.empty:
            print("Sorry, no matching restaurants found. Try a different query.")
        else:
            # Using to_string() ensures the full DataFrame is printed without truncation
            print(recommended_restaurants.to_string())
        print("-" * 40)

Dataset 'Dataset .csv' loaded successfully.
Data preprocessing complete.
TF-IDF feature matrix created.

--- Restaurant Recommendation System ---
Type 'quit' or 'exit' to stop.



What are you craving? (e.g., 'Japanese Sushi in Mandaluyong City'):  french food



Top 5 Recommended Restaurants for 'french food':
                   Restaurant Name         Cuisines       City  Aggregate rating  Votes  Similarity
3257                  Bonne Bouche   italian french  New Delhi               4.1    711    0.758360
4812          Chez Jerome - Q Cafe      cafe french  New Delhi               0.0      1    0.755451
7521  Le Cirque - The Leela Palace   french italian  New Delhi               3.8    199    0.729087
4993               Tokyo Mon Amour  japanese french  New Delhi               3.1     10    0.666684
9384      Restaurant Gordon Ramsay           french     London               4.7    320    0.609228
----------------------------------------



What are you craving? (e.g., 'Japanese Sushi in Mandaluyong City'):  italian



Top 5 Recommended Restaurants for 'italian':
       Restaurant Name         Cuisines       City  Aggregate rating  Votes  Similarity
3603           Foodies          italian  New Delhi               0.0      0    0.780678
3397     Da Pizza Farm          italian  New Delhi               0.0      1    0.780678
7841         Pasta Hub          italian  New Delhi               3.0     16    0.780678
3705         Sinyora's          italian  New Delhi               4.0     51    0.755554
4807  The Friends Cafe  chinese italian  New Delhi               0.0      0    0.698678
----------------------------------------



What are you craving? (e.g., 'Japanese Sushi in Mandaluyong City'):  new delhi



Top 5 Recommended Restaurants for 'new delhi':
            Restaurant Name Cuisines       City  Aggregate rating  Votes  Similarity
5387             Dragun Hut  chinese  New Delhi               0.0      1    0.609714
5886  Papa Veg Chinese Food  chinese  New Delhi               3.8     29    0.609714
3609             China Town  chinese  New Delhi               3.0      8    0.609714
6232           6 Pack Momos  chinese  New Delhi               2.8      7    0.609714
5223      Rita Chinese Food  chinese  New Delhi               3.1     10    0.609714
----------------------------------------



What are you craving? (e.g., 'Japanese Sushi in Mandaluyong City'):  bangalore



Top 5 Recommended Restaurants for 'bangalore':
                      Restaurant Name                          Cuisines       City  Aggregate rating  Votes  Similarity
742                         Communiti                       continental  Bangalore               4.2    334    0.839811
726                  Sultans of Spice              north indian mughlai  Bangalore               4.1   2416    0.813400
727  The Fatty Bao - Asian Gastro Bar                             asian  Bangalore               4.7   2369    0.762535
738                Koramangala Social              continental american  Bangalore               4.5   1288    0.754019
744                              Hoot  continental italian north indian  Bangalore               3.9    405    0.746012
----------------------------------------
